In [20]:
player_id = 0
def show_video(video="capture", size=(1280, 720), self_url="smart-tv-tester"):
    global player_id
    from IPython.display import HTML
    from textwrap import dedent
    player_id += 1
    html = dedent(r"""\
        <div id="player-%(player_id)i" style="width: %(width)spx; height: %(height)spx">
        </div>

        <script src="https://releases.flowplayer.org/js/flowplayer-3.2.12.min.js"></script>
        <script>
        flowplayer(
          "player-%(player_id)i",
          {
            src: "https://releases.flowplayer.org/swf/flowplayer-3.2.16.swf",
            wmode: "opaque",
          },
          {
            clip: {
              url: "%(video)s",
              provider: "rtmp",
              live: true,
              autoPlay: true,
              bufferLength: 1,
              scaling: 'fit',
            },
            plugins: {
              rtmp: {
                url: "https://releases.flowplayer.org/swf/flowplayer.rtmp-3.2.12.swf",
                netConnectionUrl: "rtmp://%(self_url)s/flvplayback",
              },
            },
          });
        </script>
        """) % dict(width=size[0], height=size[1], self_url=self_url, video=video, player_id=player_id)
    return HTML(html)


DEFAULT_SHADER = "/opt/stbt-one-video-source/noop.frag"
SHADER = "/var/opt/stbt/video-source/camera.frag"
SHADER_TEMPLATE = '/var/opt/stbt/video-source/stb-tester/stbt-camera.d/geo-correction.frag.jinja2'

def restart_conversion():
    import glob
    import re
    import os
    import subprocess
    import signal
    for x in glob.glob('/proc/*/cmdline'):
        with open(x) as f:
            if 'glshader' in f.read().split('\0'):
                pid = int(re.match('/proc/(.*)/cmdline', x).group(1))
                print "kill %i" % pid
                os.kill(pid, signal.SIGTERM)


def apply_params(params):
    import jinja2
    t = jinja2.Template(open(SHADER_TEMPLATE).read())
    with open(SHADER, 'w') as out:
        out.write(t.render(c=params))
    restart_conversion()

def reset_shader():
    import shutil
    shutil.copyfile(DEFAULT_SHADER, SHADER)
    restart_conversion()


def shell(cmd):
    import subprocess
    try:
        print subprocess.check_output(cmd, shell=True, stderr=subprocess.STDOUT)
    except subprocess.CalledProcessError as e:
        print e.output
        raise

In [19]:
show_video(video="raw", size=(1280/2,720/2))

UnboundLocalError: local variable 'player_id' referenced before assignment

In [4]:
restart_conversion()

kill 8


In [4]:
#shell("""git -C /var/opt/stbt/video-source clone https://github.com/stb-tester/stb-tester""")

def git(cmd):
    shell("git -C /var/opt/stbt/video-source/stb-tester %s" % cmd)

#git("remote add wmanley https://github.com/wmanley/stb-tester")
#git("fetch --all")

In [31]:
#git("branch --set-upstream-to=wmanley/camera-opengl")
git("""pull""")
#git("reset --hard wmanley/camera-opengl")

From https://github.com/wmanley/stb-tester
   a81470a..c451244  camera-opengl -> wmanley/camera-opengl
Updating a81470a..c451244
Fast-forward
 _stbt/gst_hacks.py | 10 +++-------
 1 file changed, 3 insertions(+), 7 deletions(-)



In [5]:
import sys
sys.path.append('/var/opt/stbt/video-source/stb-tester')
sys.path.append('/var/opt/stbt/video-source/stb-tester/stbt-camera.d')

In [6]:
import stbt
import _stbt
import stbt_camera_calibrate

In [11]:
import os
reload(_stbt.tv_driver)
os.environ['HOME'] = "/var/opt/stbt/video-source/"
os.environ['STBT_VIDEO_SERVER_BASE_URL'] = 'http://192.168.1.100/calibration-videos/'
shell("mkdir -p /root/.ssh && chmod 0700 /root/.ssh && cp $HOME/.ssh/* /root/.ssh")
shell("ssh -o StrictHostKeyChecking=False pi@192.168.1.206 true")
tv = stbt_camera_calibrate.tv_driver.create_from_description(
    "rpi:192.168.1.206", stbt_camera_calibrate.videos, 'mp4')

In [21]:
tv.show('chessboard')
reset_shader()
show_video(video="raw", size=(1280/2,720/2))

kill 240


In [23]:
reload(stbt_camera_calibrate)
os.environ['DBUS_SESSION_BUS_ADDRESS'] = "unix:path=/run/dbus/system_bus_socket"
dut = _stbt.core.new_device_under_test_from_config(
    "pulsevideosrc bus-name=com.stbtester.VideoSource.raw",
    "fakesink", "none", False,
    False, "identity")

with dut:
    params = stbt_camera_calibrate.calculate_chessboard_calibration_params(dut.get_frame())

apply_params({'geometric_params': params})


kill 275


In [24]:
tv.show('colours2')